### Imports and function definitions

In [1]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data"


import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
P = "C:\\Users\\Aaron Geller\\SCISS\\modulesbuilder"
sys.path.append(P)
import uvmodlib.v1 as uvmod

### USES Conf Templates

In [62]:
Template = """mesh
{  
    data  sphere ./modules/$folderName/sphere.obj
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        { 
            vec1f simUseTime  402500  | public | desc "year BCE" | widget slider | range  2500 402500
            vec1f ntAve 50
            vec1f tAveRange 20000
            vec1f CO2lim 0.2 | public | desc "red CO2" | widget slider | range  0 1
            bool showCO2 true 
            vec1f dy 0.001
        }
    }


    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/vostok.vs
                fragmentShader ./modules/$folderName/vostok.fs
                stateManagerVar __objectName__.Alpha        uv_alpha
                stateManagerVar __objectName__.simUseTime simUseTime
                stateManagerVar __objectName__.ntAve ntAve
                stateManagerVar __objectName__.tAveRange tAveRange
                stateManagerVar __objectName__.showCO2 showCO2
                stateManagerVar __objectName__.CO2lim CO2lim
                stateManagerVar __objectName__.dy dy

                parameter1f nFrames 30
                parameter2f CO2range 183.13 298.26
                parameter2f timerange 2500 402500

                texture glacierTex ./modules/$folderName/IceAge_2048.dds
                {
                    #Return 0 0 0 if trying to access a point outside of the dataset
                    wrapModeT GL_CLAMP_TO_BORDER
                    magnify GL_NEAREST
                    minify GL_NEAREST
                }
                texture CO2Tex ./modules/$folderName/vostok.png
                {
                    colorspace linear
                }  
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    #UV_CULL_MODE GL_FRONT
                    UV_BLEND_ENABLE true
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }
    
}"""

### Vostok Class

In [63]:
class Vostok():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("sphere.obj",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("vostok.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("vostok.fs",dataFolder)
        self.Scale = 1
        self.cr = 1000
    def setScale(self,Scale):
        self.Scale = Scale
    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale,
                                ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [64]:
model = Vostok(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [65]:
model.object.setcameraradius(10)
model.object.setcoord("Earth")
model.object.setname("VostokIceAges")
model.object.setguiname("/KavliLecture/Schrag/Vostok Ice Ages")
model.object.settargetradius(2000)
model.object.setscalefactor(655.0)
model.object.showatstartup(False)
model.object.custom = "bin 9999"
model.object.custom += """
    rotation
	{
		spice
		{
			frame		IAU_EARTH 
			reference	ECLIPJ2000
		}
	}"""

model.cr = 10000


modinfo.setname("Vostok Ice Ages ")
modinfo.setauthor("Aaron Geller<sup>1</sup>, Mark SubbaRao<sup>1</sup>, Daniel Schragg<sup>2</sup>, and Alex Griswald<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Harvard University .")
modinfo.cleardependencies()
modinfo.adddependency("Earth")
modinfo.setdesc("Vostok Ice core data used to animate Ice Ages")
#modinfo.setthumbnail("data/thumb.png")
modinfo.setversion("1.0")
generator.generate("VostokIceAges",[],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [6]:
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


Copy modified Shader files and reload

In [7]:
from config import Settings


In [220]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")